In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Load pre-trained models
cnn_model = load_model('cnn_model.h5')
mobilenet_model = load_model('mobilenetv2_tumor_detection_model.h5')
meta_model = joblib.load('ensemble_meta_model.joblib')

# Dictionary to map predicted labels to tumor types
class_mapping = {0: 'Glioma', 1: 'Meningioma', 2: 'no', 3: 'Pituitary'}

# Function to preprocess an image for CNN
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    image = image / 255.0
    return image

# Function to predict tumor type and confidence
def predict_tumor(image):
    preprocessed_image = preprocess_image(image)
    cnn_predictions = cnn_model.predict(np.expand_dims(preprocessed_image, axis=0))
    mobilenet_predictions = mobilenet_model.predict(np.expand_dims(preprocessed_image, axis=0))
    combined_predictions = np.concatenate((cnn_predictions, mobilenet_predictions), axis=1)
    ensemble_predicted_probability = meta_model.predict_proba(combined_predictions)
    confidence = np.max(ensemble_predicted_probability)
    predicted_class = np.argmax(ensemble_predicted_probability)
    tumor_type = class_mapping[predicted_class]
    return tumor_type, confidence

# Function to load and display image
def load_and_display_image():
    global image_path
    image_path = filedialog.askopenfilename()
    if image_path:
        image_path_entry.delete(0, tk.END)
        image_path_entry.insert(0, image_path)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img.thumbnail((250, 250))
        img = ImageTk.PhotoImage(img)
        label_image.config(image=img)
        label_image.image = img

# Function to predict tumor type and display result
def predict_and_display():
    global label_result
    global image_path
    if image_path:
        img = cv2.imread(image_path)
        tumor_type, confidence = predict_tumor(img)
        result_text = f"Predicted Tumor Type: {tumor_type}\nConfidence: {confidence:.2f}"
        label_result.config(text=result_text)
    else:
        label_result.config(text="Please load an image first.")

# Create the main window
root = tk.Tk()
root.title("Tumor Type Prediction")

# Create widgets
label_image_path = tk.Label(root, text="Enter Image Path:")
image_path_entry = tk.Entry(root)
button_load_image = tk.Button(root, text="Load Image", command=load_and_display_image)
label_image = tk.Label(root)
button_predict = tk.Button(root, text="Predict", command=predict_and_display)
label_result = tk.Label(root, text="")

# Arrange widgets
label_image_path.grid(row=0, column=0, sticky="e")
image_path_entry.grid(row=0, column=1, padx=5, pady=5)
button_load_image.grid(row=0, column=2, padx=5, pady=5)
label_image.grid(row=1, column=0, columnspan=3, padx=10, pady=10)
button_predict.grid(row=2, column=0, columnspan=3, pady=5)
label_result.grid(row=3, column=0, columnspan=3, pady=5)

root.mainloop()


1/1 [==============================] - 0s 57ms/step
